In [1]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
import pandas as pd

In [2]:
engine = create_engine("sqlite:///Global_Land_Temps.sqlite")
conn = engine.connect()

In [3]:
new_df = pd.read_csv("All Datasets/US-events-1980-2018.csv", low_memory=False)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 6 columns):
Name                                233 non-null object
Disaster                            233 non-null object
BeginDate                           233 non-null int64
EndDate                             233 non-null int64
Total_CPI_Adjusted_Cost_Millions    233 non-null float64
Deaths                              233 non-null int64
dtypes: float64(1), int64(3), object(2)
memory usage: 11.0+ KB


In [4]:
Base = declarative_base()
class US_Disaster_Events(Base):
    __tablename__ = 'US_Disasters'

    id = Column(Integer, primary_key=True)
    Name = Column(Text)
    Disaster = Column(Text)
    BeginDate = Column(Integer)
    EndDate = Column(Integer)
    Total_CPI_Adjusted_Cost_Millions = Column(Float)
    Deaths = Column(Integer)
   
    def __repr__(self):
        return "id={self.id}, name={self.name}"

In [5]:
Base.metadata.create_all(engine)

In [6]:
data = new_df.to_dict(orient='records')
print(data[:5])

[{'Name': 'Texas Hail Storm (June 2018)', 'Disaster': 'Severe Storm', 'BeginDate': 20180606, 'EndDate': 20180606, 'Total_CPI_Adjusted_Cost_Millions': 1150.0, 'Deaths': 0}, {'Name': 'Central and Eastern Severe Weather (May 2018)', 'Disaster': 'Severe Storm', 'BeginDate': 20180513, 'EndDate': 20180515, 'Total_CPI_Adjusted_Cost_Millions': 1370.0, 'Deaths': 5}, {'Name': 'Central and Northeastern Severe Weather (May 2018)', 'Disaster': 'Severe Storm', 'BeginDate': 20180501, 'EndDate': 20180504, 'Total_CPI_Adjusted_Cost_Millions': 1018.4, 'Deaths': 0}, {'Name': 'Southeastern Severe Storms and Tornadoes (March 2018)', 'Disaster': 'Severe Storm', 'BeginDate': 20180318, 'EndDate': 20180321, 'Total_CPI_Adjusted_Cost_Millions': 1378.8, 'Deaths': 0}, {'Name': 'Northeast Winter Storm (March 2018)', 'Disaster': 'Winter Storm', 'BeginDate': 20180301, 'EndDate': 20180303, 'Total_CPI_Adjusted_Cost_Millions': 2216.0, 'Deaths': 9}]


In [7]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [8]:
table = sqlalchemy.Table('US_Disasters', metadata, autoload=True)

In [9]:
conn.execute(table.insert(), data)

In [10]:
conn.execute("select * from US_Disasters limit 5").fetchall()

[(1, 'Texas Hail Storm (June 2018)', 'Severe Storm', 20180606, 20180606, 1150.0, 0),
 (2, 'Central and Eastern Severe Weather (May 2018)', 'Severe Storm', 20180513, 20180515, 1370.0, 5),
 (3, 'Central and Northeastern Severe Weather (May 2018)', 'Severe Storm', 20180501, 20180504, 1018.4, 0),
 (4, 'Southeastern Severe Storms and Tornadoes (March 2018)', 'Severe Storm', 20180318, 20180321, 1378.8, 0),
 (5, 'Northeast Winter Storm (March 2018)', 'Winter Storm', 20180301, 20180303, 2216.0, 9)]